<a href="https://colab.research.google.com/github/joao-dias-25/final-project-edit/blob/main/Streaming_speed_RT_JD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aplication in Real Time to Read Carris API - group 1


1. Authentication to gcloud

In [1]:
# autentication to gcloud with login

!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=VuD7eaftet1vPn8u74ZCfkSwOSVcZK&prompt=consent&token_usage=remote&access_type=offline&code_challenge=g2tNwlf9VAF9VaqzGr0QbKSQRiltmyyjQ6gnUVxRHs0&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AanRRrvA3T_I8Wx46qS5iNP7I8Ln_BL6G1r2WdJwB_RZmf69HYcNPMlSZoB3Wk7Y5KUWtQ

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [2]:
# download connector and save it local

!wget https://repo1.maven.org/maven2/com/google/cloud/bigdataoss/gcs-connector/hadoop3-2.2.7/gcs-connector-hadoop3-2.2.7-shaded.jar -P /usr/local/lib/

--2025-01-20 20:25:18--  https://repo1.maven.org/maven2/com/google/cloud/bigdataoss/gcs-connector/hadoop3-2.2.7/gcs-connector-hadoop3-2.2.7-shaded.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33831577 (32M) [application/java-archive]
Saving to: ‘/usr/local/lib/gcs-connector-hadoop3-2.2.7-shaded.jar’

gcs-connector-hadoo 100%[===================>]  32.26M  --.-KB/s    in 0.1s    

2025-01-20 20:25:18 (220 MB/s) - ‘/usr/local/lib/gcs-connector-hadoop3-2.2.7-shaded.jar’ saved [33831577/33831577]



In [3]:
# import libraries

import os
from pyspark.sql import SparkSession

#spark session
spark = SparkSession.builder \
    .appName('GCS_Spark') \
    .config('spark.jars', '/usr/local/lib/gcs-connector-hadoop3-2.2.7-shaded.jar') \
    .config('spark.hadoop.fs.gs.impl', 'com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem') \
    .getOrCreate()

# save credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/content/.config/application_default_credentials.json'

# Config PySpark to access the GCS
spark._jsc.hadoopConfiguration().set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
spark._jsc.hadoopConfiguration().set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
spark._jsc.hadoopConfiguration().set("fs.gs.auth.service.account.enable", "true")
spark._jsc.hadoopConfiguration().set("google.cloud.auth.service.account.json.keyfile", '/content/.config/application_default_credentials.json')

2. Read Stream

In [4]:
from pyspark.sql.types import *

# create schema
vehicle_schema = StructType([StructField('bearing', IntegerType(), True),
                             StructField('block_id', StringType(), True),
                             StructField('current_status', StringType(), True),
                             StructField('id', StringType(), True),
                             StructField('lat', FloatType(), True),
                             StructField('line_id', StringType(), True),
                             StructField('lon', FloatType(), True),
                             StructField('pattern_id', StringType(), True),
                             StructField('route_id', StringType(), True),
                             StructField('schedule_relationship', StringType(), True),
                             StructField('shift_id', StringType(), True),
                             StructField('speed', FloatType(), True),
                             StructField('stop_id', StringType(), True),
                             StructField('timestamp', TimestampType(), True),
                             StructField('trip_id', StringType(), True)])


#readStreaming
stream = spark.readStream.format("json").schema(vehicle_schema).load("gs://edit-de-project-streaming-data/carris-vehicles")

3. Read from API endpoint stops

In [5]:
df_stops = spark.read.option("header", "true").csv('gs://edit-data-eng-project-group1/LandingZone/GTFS/stops.txt')
df_stops = df_stops.select('stop_id','stop_lat','stop_lon')
df_stops = df_stops.withColumn("stop_lat", df_stops["stop_lat"].cast("float"))
df_stops = df_stops.withColumn("stop_lon", df_stops["stop_lon"].cast("float"))

In [6]:
df_stops.show()

+-------+---------+---------+
|stop_id| stop_lat| stop_lon|
+-------+---------+---------+
| 010001|38.754246|-8.959557|
| 010002|38.754574|-8.959615|
| 010005|38.754173|-8.961806|
| 010007|38.753197|-8.963687|
| 010008| 38.75327|-8.963504|
| 010009|38.750706|-8.962749|
| 010010|38.751003|-8.962783|
| 010011|38.748856|-8.962159|
| 010012|38.748867|-8.961929|
| 010013|38.748734|-8.966155|
| 010014|  38.7487|-8.965973|
| 010015|38.748756|-8.967106|
| 010016|38.749363|-8.966956|
| 010017|38.750122| -8.96804|
| 010018| 38.74998|-8.967987|
| 010019|38.748363|-8.970791|
| 010021|38.746655|-8.971506|
| 010023|38.747993|-8.969371|
| 010025| 38.75266|-8.961294|
| 010027|38.753258| -8.95813|
+-------+---------+---------+
only showing top 20 rows



4. Transformations

In [7]:
from pyspark.sql.functions import lag , col, coalesce, window
from pyspark.sql.window import Window
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from pyspark.sql import functions as F
import math

# Define watermarking and window duration
#watermark_duration = "90 seconds"
window_duration = "100 seconds"

#stream = stream.filter(col("timestamp").isNotNull())
# Define a window specification
#windowSpec = Window.partitionBy("id").orderBy("timestamp")

#select columns
#transform = stream.select('id', 'speed', 'timestamp','line_id','route_id','stop_id','lat', 'lon')
# Show the first few rows
#transform = transform.join(df_stops, on='stop_id', how='left')

# Create a new column 'previous_value' using lag
#transform = transform.withColumn("previous_lat", coalesce(lag("lat", 1).over(windowSpec), col('lat')))
#transform = transform.withColumn("previous_lon", coalesce(lag("lon", 1).over(windowSpec), col('lon')))

# Using aggregate functions to get "last known" data within the window

# watermark is necessary because of the aggregation
transformed = stream.withWatermark("timestamp", "60 seconds")

windowed_transform = transformed \
  .groupBy(window(transformed.timestamp, "2 minutes"), col("id")) \
  .agg(
      F.max(col("timestamp")).alias("max_ts"),
      F.last("lat", True).alias("previous_lat"),
      F.last("lon", True).alias("previous_lon"),
      F.first("lat", True).alias("lat"),
      F.first("lon", True).alias("lon")
)



# Use time-based windowing
#windowed_transform = transform.groupBy(
#    window(transform.timestamp, window_duration),
#    transform.id
#).agg(
#    F.avg("speed").alias("avg_speed"),
#    F.sum("distance").alias("total_distance")
#)


def haversine_distance(lat1, lon1, lat2, lon2):

    if any(x is None for x in [lat1, lon1, lat2, lon2]):
        return 0.0
    R = 6371  # Earth's radius in kilometers

    # Convert latitude and longitude to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Calculate differences
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Apply Haversine formula
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))

    # Calculate distance
    distance = R * c

    return distance

# Register the UDF
distance_udf = udf(haversine_distance, FloatType())

#windowed_transform = windowed_transform.withColumn("distance", distance_udf(windowed_transform["previous_lat"],windowed_transform["previous_lon"],windowed_transform["lat"],windowed_transform["lon"]))
#windowed_transform = windowed_transform.withColumn("distance_to_stop", distance_udf(windowed_transform["lat"],windowed_transform["lon"],windowed_transform["stop_lat"],windowed_transform["stop_lon"]))

# cache in memory
#transform.cache()

5. Write Stream

In [22]:
# Output function for each windowed batch
folder = 'teste_5'

def insert_windowed_vehicles(df, batch_id):
    print(f"Batch ID: {batch_id}")
    df.write.format("parquet").mode("append").save(f"gs://edit-data-eng-project-group1/datalake/stream/{folder}")


# Write the streaming query with watermark and window
windowed_query = (windowed_transform
                  .writeStream
                  .outputMode("update")
                  .foreachBatch(insert_windowed_vehicles)
                  .option('checkpointLocation', f'gs://edit-data-eng-project-group1/datalake/stream/{folder}/checkpoint')
                  #.trigger(processingTime='10 seconds')
                  .start()
)

#windowed_query.awaitTermination()

In [23]:
windowed_query.isActive

True

In [27]:
windowed_query.status

{'message': 'Writing offsets to log',
 'isDataAvailable': False,
 'isTriggerActive': True}

In [25]:
windowed_query.recentProgress

[]

In [21]:
windowed_query.stop()

In [ ]:
# Define the path to the Parquet files
parquet_path = "gs://edit-data-eng-project-group1/datalake/stream/windowed_vehicles"

# Read the Parquet files into a DataFrame
parquet_df = spark.read.parquet(parquet_path)

# Show the first few rows
parquet_df.show(truncate=False)

# Print the schema to understand the data structure
parquet_df.printSchema()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 